In [3]:
import json
import io
import re

In [4]:
with open ('../raw-files/all-party-information.json', 'r') as f:
    parties = json.load(f)['data_list']

In [5]:
parties[1]

{'id': 92,
 'id_dga': 203,
 'name_th': 'พรรคกลาง',
 'name_en': 'GLANG PARTY',
 'logo': '94.png',
 'leader': 'นายชุมพล ครุฑแก้ว',
 'secretary': 'นายสุขทวี สุวรรณชัยรบ',
 'board_list': '(กำลังปรับปรุงฐานข้อมูล) \t \t',
 'address': '127 แขวงบางยี่ขัน เขตบางพลัด กรุงเทพฯ 10700',
 'phone': '02-433-6996, 097-279-2397',
 'fax': '02-433-6997',
 'website': None,
 'policy': 'พรรคกลางนำเสนอเทคโนโลยีเพื่อประชาธิปไตย ด้วยประชาธิปไตยทางตรง อำนาจจะอยู่ในมือ(ถือ)ของประชาชนตลอดไป [ปชช.มีส่วนร่วมได้ในทุกการตัดสินใจและเป็นผู้ลงประชามติได้ปีละหลายครั้ง รวมถึงการเสนอนโยบายเองได้ด้วยปลายนิ้ว] เราไม่มีนายทุนชี้นำเบื้องหลัง เราเป็นจิตอาสาจากหลากหลายอาชีพ เสียสละเข้ามาร่วมงาน ก้าวข้ามความขัดแย้ง พัฒนาอย่างสร้างสรรค์ เน้นแก้ปัญหาอย่างยั่งยืน มุ่งสร้างความเที่ยงธรรมในสังคม และ ส่งเสริมการมีส่วนร่วม/รับฟัง/กระจายอำนาจ ส.ส.พรรคกลางจะทำงานต่อเนื่องช่วยเหลือ ปชช. ผ่านศูนย์ร้องเรียน ที่จะช่วยให้คำปรึกษาด้านกฎหมายด้วยเครือข่ายจิตอาสา ส.ส.พรรคกลางจะสอบถามความเห็นสมาชิก/ปชช. ก่อนเข้าไปอภิปราย/ออกเสียงในสภา “ปชช.ทุกคนเป็

In [4]:
'we have %d parties' % (len(parties))

'we have 104 parties'

In [5]:
party2candidates = {}
with open ('../raw-files/pm-candidates.json', 'r') as f:
    candidates = json.load(f)['Result']
print("We have %d PM candidates" % len(candidates))
for c in candidates:
    pno = c['PartyNumber']
    if pno not in party2candidates.keys():
        party2candidates[pno] = [c]
    else:
        party2candidates[pno].append(c)

We have 69 PM candidates


In [6]:
party2candidates[206]

[{'Sequence': 1,
  'Title': 'นาย',
  'FirstName': 'พงศา',
  'LastName': 'ชูแนม',
  'PartyNumber': 206,
  'PartyName': 'กรีน'}]

In [7]:
def write_line(s, template):
    s.write('{}\n\n'.format(template))
    
def parties_md_parties(f, n=5):

    for i, p in enumerate(parties[:n]):
        if n is not None and n < 10:
            print(p)
        write_func = lambda t: write_line(f, t.format(**p))
#         f.write(.format(**p))
        write_func('## %d {name_th} ({name_en})' % (i+1))
        write_func('#### หัวหน้าพรรค {leader}')
        write_func('**Donation code:** {donate_code}')
        write_func('**Address:** {address}')
        write_func('**Website:** {website}')
        write_func('**Policy**\n```\n{policy}\n```')

        if p['id_dga'] in party2candidates.keys():
            write_func('**PM Candidates**')
            for c in sorted(party2candidates[p['id_dga']], key=lambda x: x['Sequence']):
                write_func('{Sequence}. {Title} {FirstName} {LastName}'.format(**c))
        write_func('')
sio = io.StringIO()
parties_md_parties(sio, n=None)

In [8]:
with open('../raw-files/all-party-infomation-with-pm-candidates.md', 'w') as f:
    f.write(sio.getvalue())

In [9]:
def merge_party_info():
    results = []
    
    for p in parties:
        p = dict(**p)
        if p['id_dga'] in party2candidates.keys():
            pm_candidates = list(sorted(party2candidates[p['id_dga']], key=lambda x: x['Sequence']))
        else:
            pm_candidates = []
            print('no id_dga key for key=%s (%s)' % (p['id_dga'], p['name_th']))
        p['pm_candidates'] = pm_candidates
        p['slug'] = p['name_th'].replace('พรรค', '').strip()
        p['terminated'] = p['id_dga'] == 77
        results.append(p)
    return results

party_info_merged = merge_party_info()

no id_dga key for key=22 (พรรคเครือข่ายชาวนาแห่งประเทศไทย)
no id_dga key for key=158 (พรรคเงินเดือนประชาชน)
no id_dga key for key=152 (พรรคชาติไทยสามัคคี)
no id_dga key for key=141 (พรรคชาติประชาธิปไตยก้าวหน้า)
no id_dga key for key=129 (พรรคถิ่นกาขาวชาววิไล)
no id_dga key for key=53 (พรรคแทนคุณแผ่นดิน)
no id_dga key for key=96 (พรรคไทยมหารัฐพัฒนา)
no id_dga key for key=77 (พรรคไทยรักษาชาติ)
no id_dga key for key=194 (พรรคไทยรุ่งเรือง)
no id_dga key for key=144 (พรรคไทรักธรรม)
no id_dga key for key=59 (พรรคธรรมาภิบาลสังคม)
no id_dga key for key=160 (พรรคปฏิรูปไทย)
no id_dga key for key=176 (พรรคประชาชนปฏิรูป)
no id_dga key for key=205 (พรรคประชาไทย)
no id_dga key for key=73 (พรรคประชาธรรม)
no id_dga key for key=169 (พรรคประชาธิปไตยเพื่อประชาชน)
no id_dga key for key=91 (พรรคประชาสามัคคี)
no id_dga key for key=164 (พรรคผึ้งหลวง)
no id_dga key for key=163 (พรรคพลังเกษตรกรไทย)
no id_dga key for key=82 (พรรคพลังคนกีฬา)
no id_dga key for key=210 (พรรคพลังครูไทย)
no id_dga key for key=127 (พ

In [10]:
with open('../data/party-information.json', 'w') as f:
    json.dump(party_info_merged, f, ensure_ascii=False)

In [11]:
party_info_merged[0]

{'id': 95,
 'id_dga': 206,
 'name_th': 'พรรคกรีน',
 'name_en': 'Green Party',
 'logo': '97.png',
 'leader': 'นายพงศา ชูแนม',
 'secretary': 'นายวีระพงศ์ สกล',
 'board_list': '(กำลังปรับปรุงฐานข้อมูล) \t \t',
 'address': '699 ซ.จรัลสนิทวงศ์ 12 วัดท่าพระ บางกอกใหญ่ กทม. 10600',
 'phone': '081-894-1973',
 'fax': '',
 'website': None,
 'policy': 'พรรคกรีนเป็นพรรคสากลลำดับที่ 64 ของโลกมีนโยบายสอดคล้องกันทั้ง 4 ข้อ คือ \n\n1) ต้นไม้เป็นทรัพย์สิน : ออกกฎหมายรับรองต้นไม้ให้เป็นหลักทรัพย์โดยการออกบัตรคนรวย TAC ; tree asset card \n\n2) ที่ทำกินถูกต้องเป็นธรรม : คือการออกมาตรการให้ที่ดินทำกินให้ชอบด้วยกฎหมายแก้ปัญหาคนในเขตป่า 17 ล้านคนด้วยแนวทางให้สิทธิสีเขียว right&green \n\n3) เกษตรกรรมสุขภาพ : เพื่อคุ้มครองสิทธิเกษตรกรที่ผลิตอาหารสุขภาพให้มีความเท่าทัน ตามอุดมการณ์ ลดอำนาจรัฐ ตัดอำนาจทุน หนุนอำนาจประชาชน \n\n4) สันติภาพยั่งยืน : การให้สภาประชาชนในโลกดิจิทัลตรวจสอบกำกับรัฐและทุน เช่นการใช้สมการราคากำกับสินค้าให้สัมพันธ์กับราคาพืชผล',
 'donate_code': '206',
 'value': 95,
 'label': 'พรรคกรีน',
 'p

In [6]:
partiesDict = dict(map(lambda x: (x['name_th'].replace('พรรค',''), x), parties))

In [7]:
partiesDict['พลังประชารัฐ']

{'id': 81,
 'id_dga': 192,
 'name_th': 'พรรคพลังประชารัฐ',
 'name_en': 'PALANG PRACHARATH PARTY',
 'logo': '83.png',
 'leader': 'นายอุตตม สาวนายน',
 'secretary': 'นายสนธิรัตน์ สนธิจิรวงศ์',
 'board_list': '(กำลังปรับปรุงฐานข้อมูล) \t \t',
 'address': '130/1 ซ.รัชดาภิเษก 54 ถ.รัชดาภิเษก แขวงลาดยาว เขตจตุจักร กทม. 10900',
 'phone': '093-596-1526',
 'fax': '',
 'website': None,
 'policy': '(1) สวัสดิการประชารัฐ เพื่อขจัดความเหลื่อมล้ำผ่านบัตรประชารัฐ การให้สวัสดิการรายกลุ่ม ลดหนี้สร้างเงินออม และช่วยเหลือด้านที่อยู่อาศัยและที่ทำกิน \n\n(2) สังคมประชารัฐ เพื่อสร้างสังคมสงบสุข เข้มแข็งแบ่งปัน ผ่านการพัฒนา การศึกษา การกระจายความเจริญ การสร้างเมืองอัจฉริยะสีเขียว การพัฒนาชุมชนสีขาว กรุงเทพ 5.0 \n\n(3) เศรษฐกิจประชารัฐ เพื่อสร้างความสามารถและโอกาสที่เท่าเทียม ผ่านการยกระดับความสามารถผู้ผลิต การเกษตรประชารัฐ การท่องเที่ยวชุมชนและเศรษฐกิจสร้างสรรค์ เศรษฐกิจชีวภาพ การพัฒนาโครงสร้างพื้นฐานแห่งอนาคต และการลดอุปสรรคในการดำเนินธุรกิจ',
 'donate_code': '192',
 'value': 81,
 'label': 'พรรคพลังประชารัฐ'